In [1]:
import pandas as pd
import numpy as np
import os


In [5]:
json_files= [
    "data/compoundV2_transactions_ethereum_chunk_0.json",
    "data/compoundV2_transactions_ethereum_chunk_1.json",
    "data/compoundV2_transactions_ethereum_chunk_2.json"
]


In [ ]:
import os
import json
import pandas as pd

def load_and_flatten_json(file_path):
    """Load a single nested JSON file and flatten it into a list of DataFrames."""
    with open(file_path, "r") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print(f"[✗] Skipping {file_path}: Invalid JSON.")
            return []

    frames = []
    for action_type, records in data.items():
        if isinstance(records, list) and len(records) > 0:
            df = pd.json_normalize(records)
            df['action'] = action_type
            frames.append(df)
    return frames

# Step 1: Collect all JSON file paths in the folder
data_folder = "data"
json_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith(".json")]

all_frames = []

# Step 2: Process each file and extract action records
for file in json_files:
    print(f"[→] Processing: {file}")
    file_frames = load_and_flatten_json(file)
    if file_frames:
        all_frames.extend(file_frames)

# Step 3: Combine all extracted records
if all_frames:
    combined_df = pd.concat(all_frames, ignore_index=True)

    # Optional column cleanup
    combined_df = combined_df.rename(columns={
        'account.id': 'wallet_address',
        'amountUSD': 'amount_usd',
        'amount': 'amount',
        'asset.symbol': 'symbol',
        'asset.id': 'asset_id',
        'hash': 'tx_hash',
        'timestamp': 'timestamp'
    })

    # ✅ Preview the data
    print("✅ Combined Shape:", combined_df.shape)
    print(combined_df[['wallet_address', 'action', 'amount', 'amount_usd', 'symbol']].head())

    # 💾 Save to CSV
    combined_df.to_csv("data/compound_combined.csv", index=False)
    print("✅ Saved: data/compound_combined.csv")

else:
    print("⚠️ No valid data found in the JSON files.")


[→] Processing: data\compoundV2_transactions_ethereum_chunk_0.json
[→] Processing: data\compoundV2_transactions_ethereum_chunk_1.json
[→] Processing: data\compoundV2_transactions_ethereum_chunk_2.json
✅ Combined Shape: (140737, 11)
                               wallet_address    action  \
0  0x502cb8985b2c92a8d4bf309cdaa89de9be442708  deposits   
1  0xc8355d0e2c265b2fe495ebbc0fc9ad992b40dc8f  deposits   
2  0x502cb8985b2c92a8d4bf309cdaa89de9be442708  deposits   
3  0x502cb8985b2c92a8d4bf309cdaa89de9be442708  deposits   
4  0x502cb8985b2c92a8d4bf309cdaa89de9be442708  deposits   

                   amount                           amount_usd symbol  
0       10000000000000000               1.78010000000000014408    ETH  
1       80000000000000000              14.24080000000000115264    ETH  
2  4582920657824222640301  1562.320140205329756474017145956277    BAT  
3  4013962940167102266494              4013.962940167102266494    DAI  
4     9990000000000000000             1775.4727500000

In [13]:
df = pd.read_csv("data/compound_combined.csv")
print("Initial shape:", df.shape)

Initial shape: (140737, 11)


In [14]:
df = df.dropna(subset=['wallet_address', 'action', 'amount'])
df['amount_usd'] = pd.to_numeric(df['amount_usd'], errors='coerce')
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')


In [15]:
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')


In [16]:
df['amount_eth'] = df['amount'] / 1e18


In [17]:
agg_df = df.groupby(['wallet_address', 'action']).agg(
    total_amount_eth=('amount_eth', 'sum'),
    avg_amount_eth=('amount_eth', 'mean'),
    tx_count=('amount_eth', 'count'),
    first_tx=('timestamp', 'min'),
    last_tx=('timestamp', 'max')
).reset_index()


In [18]:
wallet_pivot = agg_df.pivot(index='wallet_address', columns='action', values=['total_amount_eth', 'avg_amount_eth', 'tx_count'])
wallet_pivot.columns = ['_'.join(col).strip() for col in wallet_pivot.columns]
wallet_pivot = wallet_pivot.fillna(0).reset_index()


In [19]:
wallet_times = df.groupby('wallet_address').agg(
    first_tx=('timestamp', 'min'),
    last_tx=('timestamp', 'max')
).reset_index()
wallet_times['active_days'] = (wallet_times['last_tx'] - wallet_times['first_tx']).dt.days + 1


In [20]:
wallet_features = pd.merge(wallet_pivot, wallet_times[['wallet_address', 'active_days']], on='wallet_address', how='left')


In [21]:
wallet_features['borrow_to_deposit_ratio'] = (
    wallet_features.get('total_amount_eth_borrows', 0) / (wallet_features.get('total_amount_eth_deposits', 1))
)

wallet_features['repay_to_borrow_ratio'] = (
    wallet_features.get('total_amount_eth_repays', 0) / (wallet_features.get('total_amount_eth_borrows', 1))
)


In [22]:
wallet_features['tx_total'] = wallet_features[[col for col in wallet_features.columns if col.startswith("tx_count_")]].sum(axis=1)


In [23]:
print("Final wallet-level features shape:", wallet_features.shape)
wallet_features.head()


Final wallet-level features shape: (13484, 17)


,wallet_address,total_amount_eth_borrows,total_amount_eth_deposits,total_amount_eth_repays,total_amount_eth_withdraws,avg_amount_eth_borrows,avg_amount_eth_deposits,avg_amount_eth_repays,avg_amount_eth_withdraws,tx_count_borrows,tx_count_deposits,tx_count_repays,tx_count_withdraws,active_days,borrow_to_deposit_ratio,repay_to_borrow_ratio,tx_total
0,0x00000000af5a61acaf76190794e3fdf1289288a1,0.00,0.000000,3.513533e+04,28098.247558,0.00000,0.000000,717.047457,702.456189,0.0,0.0,49.0,40.0,1,NaN,inf,89.0
1,0x00000000b1786c9698c160d78232c78d6f6474fe,0.00,0.000000,4.228557e+03,0.000000,0.00000,0.000000,151.019876,0.000000,0.0,0.0,28.0,0.0,1,NaN,inf,28.0
2,0x000000aaee6a496aaf7b7452518781786313400f,1116658.63,710200.963118,1.115716e+06,811868.831903,7112.47535,9344.749515,8084.900285,5412.458879,157.0,76.0,138.0,150.0,1,1.572314,0.999156,521.0
3,0x00000a6bbb35ea3f6f8dbc94801738f490c30a8d,0.00,6.001887,0.000000e+00,6.041512,0.00000,3.000943,0.000000,3.020756,0.0,2.0,0.0,2.0,1,0.000000,NaN,4.0
4,0x000206732258d7511fa624127228e6a032718e62,0.00,0.000000,0.000000e+00,256.800571,0.00000,0.000000,0.000000,256.800571,0.0,0.0,0.0,1.0,1,NaN,NaN,1.0


In [24]:
wallet_features.to_csv("data/final_wallet_features.csv", index=False)
